### Integrantes
- (0118135) Jose Blando
- (0042478) Rodrigo Dominguez

### Introducción

En este trabajo práctico, estaremos utilizando las acciones de Nvidia desde enero del 2017 al día de la fecha.

Vamos a intentar explicar el valor de las acciones con las aproximaciones aprendidas durante la cursada, así también como intentar explicar los cambios en la valuación de Nvidia con diferentes aspectos socioeconómicos

### Obtención de datos

Obtendremos los valores de las acciones de Nvidia usando la API de yfinance

In [42]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.models.formatters import DatetimeTickFormatter

import warnings

# Ignorar advertencias específicas
warnings.filterwarnings("ignore", category=FutureWarning, message="The behavior of DatetimeProperties.to_pydatetime is deprecated")
warnings.filterwarnings("ignore", category=UserWarning, message="no explicit representation of timezones available for np.datetime64")



In [43]:
# Obtener datos de acciones de NVIDIA desde enero de 2019
data = yf.download("NVDA", start="2017-01-01")

[*********************100%***********************]  1 of 1 completed


In [44]:
data.index = pd.to_datetime(data.index)

In [45]:
display(data)

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,NVDA,NVDA,NVDA,NVDA,NVDA,NVDA
Date,,,,,,
2017-01-03 00:00:00+00:00,2.513068,2.550250,2.659250,2.484500,2.610000,1501996000
2017-01-04 00:00:00+00:00,2.571700,2.609750,2.637500,2.538250,2.585000,1199220000
2017-01-05 00:00:00+00:00,2.506416,2.543500,2.645500,2.526250,2.613250,984296000
2017-01-06 00:00:00+00:00,2.539920,2.577500,2.606250,2.530000,2.571250,822856000
2017-01-09 00:00:00+00:00,2.642896,2.682000,2.700000,2.587500,2.587500,916248000
...,...,...,...,...,...,...
2024-11-04 00:00:00+00:00,136.050003,136.050003,138.960007,135.570007,137.210007,187528200
2024-11-05 00:00:00+00:00,139.910004,139.910004,140.369995,137.330002,137.449997,160537400


In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1977 entries, 2017-01-03 00:00:00+00:00 to 2024-11-08 00:00:00+00:00
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   (Adj Close, NVDA)  1977 non-null   float64
 1   (Close, NVDA)      1977 non-null   float64
 2   (High, NVDA)       1977 non-null   float64
 3   (Low, NVDA)        1977 non-null   float64
 4   (Open, NVDA)       1977 non-null   float64
 5   (Volume, NVDA)     1977 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 108.1 KB


Primero vamos a ver la variación del precio de cierre ajustado (Adjuste Close) a lo largo del tiempo por día (un punto por cada día desde 2019-01-01).

Adjusted Close es el precio de cierre ajustado de la acción, que toma en cuenta dividendos, divisiones de acciones o cualquier cambio en la estructura de capital de la empresa. Representa el valor `real` de la acción, considerando estos factores.

In [47]:
df = data['Close']
df = df.reset_index()

# Renombrar las columnas
df.columns = ['Fecha', 'Precio']

# Configurar Bokeh para Jupyter
output_notebook()

# Crear la fuente de datos para Bokeh
source = ColumnDataSource(df)

# Crear la figura
p = figure(x_axis_type="datetime", title="Precio Diario de Acciones",
           height=600, width=1700)

# Añadir la línea de precios
p.line(x='Fecha', y='Precio', source=source, line_width=2, color="navy", legend_label="Precio")

# Formatear el eje x para que muestre fechas adecuadamente
p.xaxis.formatter = DatetimeTickFormatter(days="%Y-%m-%d", months="%Y-%m", years="%Y")

# Añadir herramienta de hover para mostrar detalles
hover = HoverTool(
    tooltips=[
        ("Fecha", "@Fecha{%F}"),
        ("Precio", "@{Precio}{0.2f}")
    ],
    formatters={
        '@Fecha': 'datetime'
    },
    mode='vline'
)
p.add_tools(hover)

# Configuración del gráfico
p.legend.location = "top_left"
p.xaxis.axis_label = "Fecha"
p.yaxis.axis_label = "Precio"
p.legend.click_policy = "hide"

# Mostrar el gráfico
show(p)

Loading BokehJS ...

Ahora lo que vamos a hacer para suavizar la curva, y para no trabajar con tantos puntos, es agrupar los datos por semana utilizando la media.

In [48]:
# Agrupar por semanas y calcular el promedio semanal
data_weekly = data['Close'].resample('W').mean()
data_weekly.index = pd.to_datetime(data_weekly.index)
data_weekly = data_weekly.reset_index()

# Visualizar los primeros datos
data_weekly.head()

Ticker,Date,NVDA
0,2017-01-08 00:00:00+00:00,2.570250
1,2017-01-15 00:00:00+00:00,2.628900
2,2017-01-22 00:00:00+00:00,2.582687
3,2017-01-29 00:00:00+00:00,2.708150
4,2017-02-05 00:00:00+00:00,2.814600


### Visualización de los datos agrupados por semana.

In [49]:
# Renombrar las columnas
data_weekly.columns = ['Fecha', 'Precio']
df = data_weekly

# Renombrar las columnas
df.columns = ['Fecha', 'Precio']

# Configurar Bokeh para Jupyter
output_notebook()

# Crear la fuente de datos para Bokeh
source = ColumnDataSource(df)

# Crear la figura
p = figure(x_axis_type="datetime", title="Precio Diario de Acciones",
           height=600, width=1700)

# Añadir la línea de precios
p.line(x='Fecha', y='Precio', source=source, line_width=2, color="navy", legend_label="Precio")

# Formatear el eje x para que muestre fechas adecuadamente
p.xaxis.formatter = DatetimeTickFormatter(days="%Y-%m-%d", months="%Y-%m", years="%Y")

# Añadir herramienta de hover para mostrar detalles
hover = HoverTool(
    tooltips=[
        ("Fecha", "@Fecha{%F}"),
        ("Precio", "@{Precio}{0.2f}")
    ],
    formatters={
        '@Fecha': 'datetime'
    },
    mode='vline'
)
p.add_tools(hover)

# Configuración del gráfico
p.legend.location = "top_left"
p.xaxis.axis_label = "Fecha"
p.yaxis.axis_label = "Precio"
p.legend.click_policy = "hide"

# Mostrar el gráfico
show(p)

Loading BokehJS ...

# Aproximación de la curva por mínimos cuadrados

La siguiente es una clase que permite realizar aproximaciones por mínimos cuadrados a un conjunto de datos. 
La clase recibe un DataFrame con los datos, el nombre de la columna que contiene las fechas y el nombre de la columna que contiene los valores de la variable dependiente. 
Luego, se pueden calcular las aproximaciones lineal, exponencial y potencial, y determinar cuál es la mejor en función del error cuadrático medio (MSE).

In [50]:
import numpy as np
import pandas as pd

class LeastSquaresApproximation:
    """
    Clase para realizar aproximaciones por mínimos cuadrados a un conjunto de datos.
    
    Parameters:
        df (pandas.DataFrame): DataFrame con los datos
        x_column (str): Nombre de la columna que contiene las fechas
        y_column (str): Nombre de la columna que contiene los valores de la variable dependiente
    
    Examples:
    ```
    # Crear instancia de la clase con la sección de datos
    approximator = LeastSquaresApproximation(section)
    
    # Calcular las aproximaciones
    approximations = approximator.calculate_approximations()
    
    # Para ver solo las aproximaciones:
    approximator.plot_approximations(with_curve=False)
    
    # Para ver las aproximaciones junto con la curva real:
    approximator.plot_approximations(with_curve=True)
    
    # Acceder a los resultados guardados
    best_fit = approximations['best_fit']
    linear_approx = approximations['linear']
    exponential_approx = approximations['exponential']
    power_approx = approximations['power']
    ```
    """
    
    def __init__(self, df, x_column='Fecha', y_column='Precio'):
        """
        Inicializa la clase con un DataFrame y los nombres de las columnas de entrada y salida.

        Parameters:
        df (pandas.DataFrame): DataFrame con los datos
        x_column (str): Nombre de la columna que contiene las fechas
        y_column (str): Nombre de la columna que contiene los valores de la variable dependiente
        """
        self.df = df.copy() # Copiar el DataFrame para evitar modificar el original
        self.x_column = x_column # Nombre de la columna x
        self.y_column = y_column # Nombre de la columna y

        # Convertir fechas a números (días desde el inicio)
        self.start_date = pd.to_datetime(self.df[x_column].min()) # Primer X
        self.end_date = pd.to_datetime(self.df[x_column].max()) # Ultimo X
        self.x_days = (pd.to_datetime(self.df[x_column]) - self.start_date).dt.days # A cada fecha le restamos la primera fecha
        self.y = self.df[y_column].values # Todos los valores de Y

        # Crear array de días para las aproximaciones
        self.x_days_continuous = np.arange(self.x_days.min(), self.x_days.max() + 1) # Rango de días continuos
        self.dates_continuous = pd.date_range(start=self.start_date, end=self.end_date, freq='D') # Fechas correspondientes

        # Variable para almacenar las aproximaciones
        self.approximations = None

    def calculate_approximations(self):
        """
        Calcula todas las aproximaciones (lineal, exponencial, y potencial) y determina la mejor
        en función del error cuadrático medio (MSE).

        Returns:
        dict: Diccionario con las aproximaciones lineal, exponencial, y potencial, incluyendo
              sus parámetros, ecuaciones y errores MSE, además de la mejor aproximación.
        """
        linear = self._linear_approximation()
        exponential = self._exponential_approximation()
        power = self._power_approximation()

        self.approximations = {
            'linear': linear,
            'exponential': exponential,
            'power': power,
            'best_fit': min([linear, exponential, power], key=lambda x: x['error'])
        }

        # Imprimir resultados
        print("Resultados de las aproximaciones:")
        print("\nAproximación Lineal:")
        print(f"Ecuación: {linear['equation']}")
        print(f"Error (MSE): {linear['error']:.2f}")

        print("\nAproximación Exponencial:")
        print(f"Ecuación: {exponential['equation']}")
        print(f"Error (MSE): {exponential['error']:.2f}")

        print("\nAproximación Potencial:")
        print(f"Ecuación: {power['equation']}")
        print(f"Error (MSE): {power['error']:.2f}")

        print(f"\nMejor aproximación: {self.approximations['best_fit']['type'].capitalize()}")
        print(f"Ecuación: {self.approximations['best_fit']['equation']}")
        print(f"Error (MSE): {self.approximations['best_fit']['error']:.2f}")

        return self.approximations

    def _linear_approximation(self):
        """
        Calcula la aproximación lineal de la forma y = mx + b.

        Returns:
        dict: Diccionario con la ecuación lineal, parámetros (pendiente m y intercepto b),
              predicciones, y el error cuadrático medio.
        """
        x = self.x_days # Días
        y = self.y # Precios
        n = len(x) # Cantidad de puntos

        sum_x = np.sum(x) # Sumatoria de x
        sum_y = np.sum(y) # Sumatoria de y
        sum_xy = np.sum(x * y) # Sumatoria de x * y
        sum_x2 = np.sum(x**2) # Sumatoria de x^2

        m = (n * sum_xy - sum_x * sum_y) / (n * sum_x2 - sum_x**2) # Pendiente
        b = (sum_y - m * sum_x) / n # Intercepto
        linear_pred = m * self.x_days_continuous + b # Predicción
        linear_error = self._calculate_error(y, m * x + b) # Error cuadrático medio

        return {
            'type': 'linear',
            'parameters': {'m': m, 'b': b},
            'y_pred': linear_pred,
            'error': linear_error,
            'equation': f'y = {m:.4f}x + {b:.4f}'
        }

    def _exponential_approximation(self):
        """
        Calcula la aproximación exponencial de la forma y = a * e^(bx).

        Returns:
        dict: Diccionario con la ecuación exponencial, parámetros (a y b),
              predicciones, y el error cuadrático medio.
        """
        x = self.x_days # Días
        y = self.y # Precios
        n = len(x) # Cantidad de puntos

        log_y = np.log(y) # Logaritmo natural de y
        sum_log_y = np.sum(log_y) # Sumatoria de log(y)
        sum_x_log_y = np.sum(x * log_y) # Sumatoria de x * log(y)
        sum_x = np.sum(x) # Sumatoria de x
        sum_x2 = np.sum(x**2) # Sumatoria de x^2

        b_exp = (n * sum_x_log_y - sum_x * sum_log_y) / (n * sum_x2 - sum_x**2) # Pendiente
        log_a_exp = (sum_log_y - b_exp * sum_x) / n # Logaritmo natural del coeficiente a
        a_exp = np.exp(log_a_exp) # Coeficiente a
        exp_pred = a_exp * np.exp(b_exp * self.x_days_continuous) # Predicción
        exp_error = self._calculate_error(y, a_exp * np.exp(b_exp * x)) # Error cuadrático medio

        return {
            'type': 'exponential',
            'parameters': {'a': a_exp, 'b': b_exp},
            'y_pred': exp_pred,
            'error': exp_error,
            'equation': f'y = {a_exp:.4f}e^({b_exp:.4f}x)'
        }

    def _power_approximation(self):
        """
        Calcula la aproximación potencial de la forma y = a * x^b.

        Returns:
        dict: Diccionario con la ecuación potencial, parámetros (a y b),
              predicciones, y el error cuadrático medio.
        """
        x = self.x_days # Días
        y = self.y # Precios
        n = len(x) # Cantidad de puntos

        log_x = np.log(x + 1)  # Evitar log(0)
        log_y = np.log(y) # Logaritmo natural de y
        sum_log_x = np.sum(log_x) # Sumatoria de log(x)
        sum_log_y = np.sum(log_y) # Sumatoria de log(y)
        sum_log_x_log_y = np.sum(log_x * log_y) # Sumatoria de log(x) * log(y)
        log_x2 = np.sum(log_x**2) # Sumatoria de log(x)^2

        b_pow = (n * sum_log_x_log_y - sum_log_x * sum_log_y) / (n * log_x2 - sum_log_x**2) # Pendiente
        log_a_pow = (sum_log_y - b_pow * sum_log_x) / n # Logaritmo natural del coeficiente a
        a_pow = np.exp(log_a_pow) # Coeficiente a
        pow_pred = a_pow * (self.x_days_continuous + 1) ** b_pow # Predicción
        pow_error = self._calculate_error(y, a_pow * (x + 1) ** b_pow) # Error cuadrático medio

        return {
            'type': 'power',
            'parameters': {'a': a_pow, 'b': b_pow},
            'y_pred': pow_pred,
            'error': pow_error,
            'equation': f'y = {a_pow:.4f}x^{b_pow:.4f}'
        }    

    def _calculate_error(self, y_real, y_pred):
        """
        Calcula el error cuadrático medio (MSE) entre los valores reales y los predichos.

        Parameters:
        y_real (np.array): Valores reales de y
        y_pred (np.array): Valores predichos de y

        Returns:
        float: El error cuadrático medio entre los valores reales y los predichos.
        """
        return np.mean((y_real - y_pred) ** 2) # Promedio de las diferencias al cuadrado

    def plot_approximations(self, with_curve=False):
        """
        Grafica las aproximaciones calculadas de manera interactiva con Bokeh.

        Parameters:
        with_curve (bool): Si True, incluye la curva que une los puntos reales.
        """
        if self.approximations is None:
            print("Primero debes calcular las aproximaciones usando calculate_approximations()")
            return

        # Convertir las fechas a datetime de Python para evitar advertencias de zonas horarias
        original_dates = pd.to_datetime(self.df[self.x_column]).dt.to_pydatetime()
        continuous_dates = self.dates_continuous.to_pydatetime()

        # Configurar la figura de Bokeh
        p = figure(width=1700, height=600, x_axis_type="datetime", title="Aproximaciones por Mínimos Cuadrados",
                   tools="pan,wheel_zoom,box_zoom,reset,save")

        # Graficar puntos originales
        p.scatter(original_dates, self.y, size=6, color="black", alpha=0.6, legend_label="Datos reales")

        # Graficar la curva que une los puntos si se solicita
        if with_curve:
            p.line(original_dates, self.y, color="gray", line_dash="dotted", legend_label="Curva real")

        # Colores para cada tipo de aproximación
        colors = {
            'linear': 'red',
            'exponential': 'green',
            'power': 'blue'
        }

        # Graficar cada aproximación
        for approx_type, color in colors.items():
            approx = self.approximations[approx_type]
            p.line(continuous_dates, approx['y_pred'], color=color, line_width=2,
                   legend_label=f"{approx['type'].capitalize()} - {approx['equation']}",
                   line_dash="solid")

        # Añadir HoverTool para mostrar detalles de los puntos
        hover = HoverTool(
            tooltips=[
                ("Fecha", "@x{%F}"),
                ("Precio predicho", "@y"),
            ],
            formatters={
                '@x': 'datetime',  # Formateo de la fecha
            },
            mode='vline'
        )
        p.add_tools(hover)

        # Configurar leyendas y etiquetas
        p.legend.location = "top_left"
        p.xaxis.axis_label = "Fecha"
        p.yaxis.axis_label = "Precio"
        p.legend.click_policy = "hide"  # Permitir ocultar líneas al hacer clic en la leyenda

        # Mostrar la figura
        show(p)

    # def plot_approximations(self, with_curve=False, figsize=(15, 10)):
    #     """
    #     Grafica las aproximaciones calculadas.
    # 
    #     Parameters:
    #     with_curve (bool): Si True, incluye la curva que une los puntos reales
    #     figsize (tuple): Tamaño de la figura (ancho, alto)
    #     """
    #     if self.approximations is None:
    #         print("Primero debes calcular las aproximaciones usando calculate_approximations()")
    #         return
    # 
    #     plt.figure(figsize=figsize)
    # 
    #     # Graficar puntos originales
    #     plt.scatter(pd.to_datetime(self.df[self.x_column]), self.y, color='black', alpha=0.5, label='Datos reales', s=20)
    # 
    #     # Graficar la curva que une los puntos si se solicita
    #     if with_curve:
    #         plt.plot(pd.to_datetime(self.df[self.x_column]), self.y, 'gray', alpha=0.5, label='Curva real', linewidth=1)
    # 
    #     # Graficar aproximaciones
    #     colors = {
    #         'linear': 'red',
    #         'exponential': 'green',
    #         'power': 'blue'
    #     }
    # 
    #     for approx_type, color in colors.items():
    #         approx = self.approximations[approx_type]
    #         plt.plot(self.dates_continuous, approx['y_pred'], color=color,
    #                  label=f"{approx['type'].capitalize()}\n{approx['equation']}\nMSE: {approx['error']:.2f}")
    # 
    #     plt.xlabel('Fecha')
    #     plt.ylabel('Precio')
    #     title = 'Aproximaciones por Mínimos Cuadrados'
    #     if with_curve:
    #         title += ' con Curva Real'
    #     plt.title(title)
    #     plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    #     plt.grid(True)
    #     plt.xticks(rotation=45)
    #     plt.tight_layout()
    #     plt.show()

In [51]:
def plot_stock_price(df):
    """
    Genera un gráfico interactivo de Bokeh a partir de un DataFrame de pandas.
    
    Parámetros:
    - df: pd.DataFrame que contiene las columnas 'Fecha' y 'Precio' para generar el gráfico.
    
    Retorno:
    - Muestra el gráfico de Bokeh en Jupyter Notebook.
    """
    # Configurar Bokeh para Jupyter
    output_notebook()

    # Crear la fuente de datos para Bokeh
    source = ColumnDataSource(df)

    # Crear la figura
    p = figure(x_axis_type="datetime", title="Precio Diario de Acciones", height=600, width=1700)

    # Añadir la línea de precios
    p.line(x='Fecha', y='Precio', source=source, line_width=2, color="navy", legend_label="Precio")

    # Formatear el eje x para que muestre fechas adecuadamente
    p.xaxis.formatter = DatetimeTickFormatter(days="%Y-%m-%d", months="%Y-%m", years="%Y")

    # Añadir herramienta de hover para mostrar detalles
    hover = HoverTool(
        tooltips=[
            ("Fecha", "@Fecha{%F}"),
            ("Precio", "@{Precio}{0.2f}")
        ],
        formatters={
            '@Fecha': 'datetime'
        },
        mode='vline'
    )
    p.add_tools(hover)

    # Configuración del gráfico
    p.legend.location = "top_left"
    p.xaxis.axis_label = "Fecha"
    p.yaxis.axis_label = "Precio"
    p.legend.click_policy = "hide"

    # Mostrar el gráfico
    show(p)


## 2017-01 a 2018-09: Boom de las Criptomonedas y Gaming

En 2018, las acciones de NVIDIA alcanzaron un alto nivel, alrededor de $7, impulsadas principalmente por dos factores:

1. Boom de las Criptomonedas: La minería de criptomonedas como Bitcoin y Ethereum fue una tendencia muy fuerte entre 2017 y principios de 2018. NVIDIA, como uno de los principales proveedores de GPUs, experimentó un aumento en la demanda de sus productos, ya que las GPUs son esenciales para el proceso de minería. Esto elevó considerablemente sus ingresos y, en consecuencia, su valoración en bolsa.

2. Crecimiento en Gaming y Centros de Datos: NVIDIA también se benefició de la creciente demanda de GPUs para gaming, inteligencia artificial y centros de datos. Su serie de tarjetas gráficas GeForce se consolidó como una de las mejores opciones para los jugadores, y sus GPUs para centros de datos comenzaron a ser muy utilizadas para tareas de inteligencia artificial y aprendizaje profundo.

In [52]:
# Seleccionar una sección de los datos
section = df[(df['Fecha'] >= '2017-01-01') & (df['Fecha'] <= '2018-09-24')]

In [53]:
plot_stock_price(section)

Loading BokehJS ...

In [54]:
# Crear instancia de la clase con la sección de datos
approximator = LeastSquaresApproximation(section)

# Calcular las aproximaciones
approximations = approximator.calculate_approximations()

Resultados de las aproximaciones:

Aproximación Lineal:
Ecuación: y = 0.0075x + 2.4030
Error (MSE): 0.11

Aproximación Exponencial:
Ecuación: y = 2.6347e^(0.0017x)
Error (MSE): 0.23

Aproximación Potencial:
Ecuación: y = 1.0824x^0.2636
Error (MSE): 0.37

Mejor aproximación: Linear
Ecuación: y = 0.0075x + 2.4030
Error (MSE): 0.11


In [55]:
# Para ver solo las aproximaciones:
approximator.plot_approximations(with_curve=False)

## 2018-10 a 2019-01: Caída Post-Boom de Criptomonedas

1. **Desplome de las Criptomonedas**: Cuando el valor de las criptomonedas se desplomó, la demanda de GPUs para minería cayó rápidamente. Esto generó un exceso de inventario, y NVIDIA experimentó una fuerte caída en ingresos y, en consecuencia, en su valor de mercado.
2. **Menor Demanda en Centros de Datos**: Aunque los centros de datos seguían en crecimiento, el ritmo desacelerado afectó a los ingresos, que dependían fuertemente de este sector.

In [56]:
# Seleccionar una sección de los datos
section = df[(df['Fecha'] >= '2018-09-25') & (df['Fecha'] <= '2018-12-24')]

In [57]:
plot_stock_price(section)

Loading BokehJS ...

In [58]:
# Crear instancia de la clase con la sección de datos
approximator = LeastSquaresApproximation(section)

# Calcular las aproximaciones
approximations = approximator.calculate_approximations()

Resultados de las aproximaciones:

Aproximación Lineal:
Ecuación: y = -0.0435x + 6.8481
Error (MSE): 0.10

Aproximación Exponencial:
Ecuación: y = 7.0431e^(-0.0087x)
Error (MSE): 0.08

Aproximación Potencial:
Ecuación: y = 8.4721x^-0.1637
Error (MSE): 0.59

Mejor aproximación: Exponential
Ecuación: y = 7.0431e^(-0.0087x)
Error (MSE): 0.08


In [59]:
# Para ver solo las aproximaciones:
approximator.plot_approximations(with_curve=False)

## 2019-02 a 2020-02: Recuperación y Diversificación

1. **Recuperación Gradual**: NVIDIA comenzó a estabilizarse y recuperar su valor de mercado gracias a la diversificación en IA, gaming y la incorporación de tecnologías de trazado de rayos (ray tracing) en sus GPUs, consolidando su dominio en gráficos de alta calidad.
2. **Expansión en AI y Centros de Datos**: La demanda de AI en industrias de alta tecnología permitió a NVIDIA recuperarse. Sus productos se volvieron esenciales en sistemas de aprendizaje profundo, visión artificial y simulaciones en centros de datos.



In [60]:
# Seleccionar una sección de los datos
section = df[(df['Fecha'] >= '2018-12-25') & (df['Fecha'] <= '2020-02-19')]
plot_stock_price(section)

Loading BokehJS ...

In [61]:
# Crear instancia de la clase con la sección de datos
approximator = LeastSquaresApproximation(section)

In [62]:
# Calcular las aproximaciones
approximations = approximator.calculate_approximations()

Resultados de las aproximaciones:

Aproximación Lineal:
Ecuación: y = 0.0059x + 3.3365
Error (MSE): 0.22

Aproximación Exponencial:
Ecuación: y = 3.4631e^(0.0012x)
Error (MSE): 0.19

Aproximación Potencial:
Ecuación: y = 2.5546x^0.1121
Error (MSE): 0.40

Mejor aproximación: Exponential
Ecuación: y = 3.4631e^(0.0012x)
Error (MSE): 0.19


In [63]:
# Para ver solo las aproximaciones:
approximator.plot_approximations(with_curve=False)

## 2020-03 a 2021-01: Pandemia y Aumento en Demanda Tecnológica
1. **COVID-19**: La pandemia cambió la dinámica laboral, y la demanda de hardware para videojuegos y computación en nube aumentó, impulsando aún más las ventas de GPUs.
2. **Escasez Global de Semiconductores**: La creciente demanda junto a problemas de producción generaron una escasez de chips, lo cual, aunque afectó el suministro, también permitió que NVIDIA mantuviera precios altos y consolidara sus ingresos.

In [64]:
# Seleccionar una sección de los datos
section = df[(df['Fecha'] >= '2020-03-24') & (df['Fecha'] <= '2021-01-29')]
plot_stock_price(section)

Loading BokehJS ...

In [65]:
# Crear instancia de la clase con la sección de datos
approximator = LeastSquaresApproximation(section)

In [66]:
# Calcular las aproximaciones
approximations = approximator.calculate_approximations()

Resultados de las aproximaciones:

Aproximación Lineal:
Ecuación: y = 0.0261x + 7.1183
Error (MSE): 0.81

Aproximación Exponencial:
Ecuación: y = 7.2713e^(0.0026x)
Error (MSE): 1.31

Aproximación Potencial:
Ecuación: y = 4.0837x^0.2066
Error (MSE): 0.86

Mejor aproximación: Linear
Ecuación: y = 0.0261x + 7.1183
Error (MSE): 0.81


In [67]:
# Para ver solo las aproximaciones:
approximator.plot_approximations(with_curve=False)

## 2021-02 a 2021-12: Demanda Post-Pandemia y Escasez Persistente
1. **Demanda de Computación en la Nube y Juegos**: Las empresas tecnológicas siguieron creciendo en la nube y en juegos, manteniendo la demanda alta de productos de NVIDIA.
2. **Escasez de Semiconductores Prolongada**: La falta de semiconductores limitó la capacidad de NVIDIA para satisfacer toda la demanda, pero también ayudó a mantener los precios altos, favoreciendo su valor de mercado.

In [68]:
# Seleccionar una sección de los datos
section = df[(df['Fecha'] >= '2021-02-01') & (df['Fecha'] <= '2021-12-31')]
plot_stock_price(section)

Loading BokehJS ...

In [69]:
# Crear instancia de la clase con la sección de datos
approximator = LeastSquaresApproximation(section)

In [70]:
# Calcular las aproximaciones
approximations = approximator.calculate_approximations()

Resultados de las aproximaciones:

Aproximación Lineal:
Ecuación: y = 0.0557x + 10.8880
Error (MSE): 4.30

Aproximación Exponencial:
Ecuación: y = 12.2333e^(0.0028x)
Error (MSE): 3.23

Aproximación Potencial:
Ecuación: y = 7.9399x^0.1849
Error (MSE): 14.24

Mejor aproximación: Exponential
Ecuación: y = 12.2333e^(0.0028x)
Error (MSE): 3.23


In [71]:
# Para ver solo las aproximaciones:
approximator.plot_approximations(with_curve=False)

## 2022-01 a 2022-10: Inflación y Ajuste en el Mercado Tecnológico

1. **Alza de Tasas de Interés**: La inflación y el aumento en las tasas de interés afectaron negativamente a las empresas tecnológicas, llevando a un ajuste en sus valoraciones.
2. **Reducción en Demanda de Criptomonedas**: La minería de criptomonedas experimentó una caída, afectando las ventas de GPUs. Aunque gaming y centros de datos se mantuvieron estables, la desaceleración económica pesó sobre el valor de NVIDIA.

In [72]:
# Seleccionar una sección de los datos
section = df[(df['Fecha'] >= '2022-01-01') & (df['Fecha'] <= '2022-10-14')]
plot_stock_price(section)

Loading BokehJS ...

In [73]:
# Crear instancia de la clase con la sección de datos
approximator = LeastSquaresApproximation(section)

In [74]:
# Calcular las aproximaciones
approximations = approximator.calculate_approximations()

Resultados de las aproximaciones:

Aproximación Lineal:
Ecuación: y = -0.0522x + 27.2409
Error (MSE): 3.97

Aproximación Exponencial:
Ecuación: y = 28.0841e^(-0.0027x)
Error (MSE): 3.80

Aproximación Potencial:
Ecuación: y = 42.5712x^-0.1711
Error (MSE): 11.11

Mejor aproximación: Exponential
Ecuación: y = 28.0841e^(-0.0027x)
Error (MSE): 3.80


In [75]:
# Para ver solo las aproximaciones:
approximator.plot_approximations(with_curve=False)

## 2023-01 en Adelante: Innovación en AI Generativa y Recuperación del Mercado

1. **IA Generativa**: NVIDIA se consolidó como un proveedor esencial para aplicaciones avanzadas de AI generativa, debido a la capacidad de sus GPUs para entrenar grandes modelos de IA, y expandió su mercado en empresas enfocadas en tecnologías como los LLMs (Modelos de Lenguaje Grande).
2. **Crecimiento en Centros de Datos e Investigación AI**: Las iniciativas en IA generativa, como el procesamiento de datos de lenguaje natural y visión artificial, impulsaron la demanda de sus GPUs. Este sector, en plena expansión, fortaleció la percepción de NVIDIA como líder en innovación, estabilizando y aumentando su valor de mercado nuevamente.

In [76]:
# Seleccionar una sección de los datos
section = df[(df['Fecha'] >= '2023-01-01') & (df['Fecha'] <= '2025-01-01')]
plot_stock_price(section)

Loading BokehJS ...

In [77]:
# Crear instancia de la clase con la sección de datos
approximator = LeastSquaresApproximation(section)

In [78]:
# Calcular las aproximaciones
approximations = approximator.calculate_approximations()

Resultados de las aproximaciones:

Aproximación Lineal:
Ecuación: y = 0.1842x + 4.0136
Error (MSE): 126.96

Aproximación Exponencial:
Ecuación: y = 19.6258e^(0.0031x)
Error (MSE): 95.33

Aproximación Potencial:
Ecuación: y = 3.3020x^0.5128
Error (MSE): 428.06

Mejor aproximación: Exponential
Ecuación: y = 19.6258e^(0.0031x)
Error (MSE): 95.33


In [79]:
# Para ver solo las aproximaciones:
approximator.plot_approximations(with_curve=False)